### Finetuning XLMRoberta for sentiment analysis focused on Burmese text with labels like
- positive
- negative
- neutral


In [1]:
from datasets import Dataset,load_dataset
import numpy as np
import pandas as pd

# load the dataset
# dataset 1
df1 = load_dataset("kalixlouiis/burmese-sentiment-analysis")["train"]
df1 = df1.to_pandas()[["text_input","output"]]

# rename column
df1.columns = ['text','label']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train_data.json: 0.00B [00:00, ?B/s]

val_data.json: 0.00B [00:00, ?B/s]

test_data.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2424 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/303 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/303 [00:00<?, ? examples/s]

In [2]:
# Dataset 2
df2 = load_dataset("chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset")["train"]
df2 = df2.to_pandas()[["Text-MM","Sentiment"]]


## Cut off extra spaces and lowercase
df2["Sentiment"] = df2["Sentiment"].str.strip().str.lower()

# sentiment unique values are 191
df2["Sentiment"].str.lower()
df2["Sentiment"].value_counts()

# only takes positive, negative and neutral
df2 = df2[df2["Sentiment"].isin(["positive","negative","neutral"])]

# rename column
df2.columns = ["text","label"]
df2.isnull().sum()

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/149k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/732 [00:00<?, ? examples/s]

,0
text,0
label,0


In [3]:
# Combine

df = pd.concat([df1,df2],ignore_index=True)
df.isnull().sum()

,0
text,0
label,0


In [6]:
# Third one
df3 = pd.read_csv("burmese_sentiment_synthetic.csv")

# rename
df3.columns=["text","label"]
df3.isnull().sum()

,0
text,0
label,0


In [7]:
df = pd.concat([df,df3],ignore_index=True)
df.isnull().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2521 entries, 0 to 2520
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2521 non-null   object
 1   label   2521 non-null   object
dtypes: object(2)
memory usage: 39.5+ KB


In [8]:
# fourth dataset
df4 = pd.read_csv("burmese_sentiment_synthetic_v2.csv")
# rename columns
df4.columns= ["text","label"]

In [9]:
df = pd.concat([df,df4],ignore_index=True)
df.head()
csv_path_new = "./burmese_sentiment_dataset.csv"
df.to_csv(csv_path_new, index=False)

In [ ]:
# Converting zawgyi to unicode
from icu import Transliterator

# For Unicode to Zawgyi, you would typically use "my-Zawgyi".
converter = Transliterator.createInstance("Zawgyi-my")

zawgyi_text = "သီဟိုဠ္မွ ဉာဏ္ႀကီးရွင္သည္ အာယုဝၯနေဆးၫႊန္းစာကို ဇလြန္ေဈးေဘး ဗာဒံပင္ထက္ အဓိ႒ာန္လ်က္ ဂဃနဏဖတ္ခဲ့သည္။"

#convert to unicode
unicode_text = converter.transliterate(zawgyi_text)

print(f"Original Zawgyi: {zawgyi_text}")
print("\n")
print(f"Converted to unicode: {unicode_text}")

Original Zawgyi: သီဟိုဠ္မွ ဉာဏ္ႀကီးရွင္သည္ အာယုဝၯနေဆးၫႊန္းစာကို ဇလြန္ေဈးေဘး ဗာဒံပင္ထက္ အဓိ႒ာန္လ်က္ ဂဃနဏဖတ္ခဲ့သည္။


Converted to unicode: သီဟိုဠ်မှ ဉာဏ်ကြီးရှင်သည် အာယုဝဍ္ဎနဆေးညွှန်းစာကို ဇလွန်ဈေးဘေး ဗာဒံပင်ထက် အဓိဋ္ဌာန်လျက် ဂဃနဏဖတ်ခဲ့သည်။


In [ ]:
# use ZawgyiDetector to detect if the text is Zawgyi or not
# >0.95 woulb be very likely => then convert to unicode
# if not don't convert to unicode again (because converting unicode to unicode again might even corrupt it or produce unexpected characters)

from myanmartools import ZawgyiDetector

detector = ZawgyiDetector()
score = detector.get_zawgyi_probability("hacker တွေ အန္တရာယ်ရှိတယ်")
print("Score for unicode text", round(score,3))
score2 = detector.get_zawgyi_probability("သီဟိုဠ္မွ ဉာဏ္ႀကီးရွင္သည္ အာယုဝၯနေဆးၫႊန္းစာကို ဇလြန္ေဈးေဘး ဗာဒံပင္ထက္ အဓိ႒ာန္လ်က္ ဂဃနဏဖတ္ခဲ့သည္။")
print("Score for zawgyi text",round(score2,3))

Score for unicode text 0.0
Score for zawgyi text 1.0


In [ ]:
def preprocess_text(text):
  score = detector.get_zawgyi_probability(text)
  score = round(score,3)
  if score> 0.95:
    unicode_text = converter.transliterate(text)
  else:
    unicode_text = text

  return unicode_text

tokens = preprocess_text("သီဟိုဠ္မွ ဉာဏ္ႀကီးရွင္သည္ အာယုဝၯနေဆးၫႊန္းစာကို ဇလြန္ေဈးေဘး ဗာဒံပင္ထက္ အဓိ႒ာန္လ်က္ ဂဃနဏဖတ္ခဲ့သည္။")
print(tokens)

tokens2 = preprocess_text("hacker တွေ အန္တရာယ်ရှိတယ်")
print(tokens2)

သီဟိုဠ်မှ ဉာဏ်ကြီးရှင်သည် အာယုဝဍ္ဎနဆေးညွှန်းစာကို ဇလွန်ဈေးဘေး ဗာဒံပင်ထက် အဓိဋ္ဌာန်လျက် ဂဃနဏဖတ်ခဲ့သည်။
hacker တွေ အန္တရာယ်ရှိတယ်


In [ ]:
# Apply to dataframe's text
df["text_cleaned"] = df["text"].apply(preprocess_text)

In [ ]:
df = df.drop('text',axis=1)
df


,label,text_cleaned
0,positive,ကားဝယ်ချင်တယ်
1,neutral,အားကစားလုပ်ချင်တယ်
2,negative,ဒီနေ့ ဘာလုပ်လုပ် အဆင်မပြေဘူး
3,negative,hacker တွေ အန္တရာယ်ရှိတယ်
4,positive,အရာအားလုံး အကောင်းဘက်ကို ဦးတည်နေတယ်
...,...,...
2606,neutral,သူက အဝတ်လျှော်နေတာ။
2607,neutral,ဒီနေ့ ကားသွားမယ်။
2608,neutral,သူက လမ်းဆုံမှာရပ်နေတယ်။
2609,neutral,မနက်ဖြန်မှာ ခရီးသွားမယ်။


In [ ]:
import numpy as np
import torch
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = "cuda" if torch.cuda.is_available() else "cpu"

label_map = {"positive": 0, "neutral": 1, "negative": 2}

# convert to integer
df["label"] = df["label"].map(label_map)
df


,label,text_cleaned
0,0,ကားဝယ်ချင်တယ်
1,1,အားကစားလုပ်ချင်တယ်
2,2,ဒီနေ့ ဘာလုပ်လုပ် အဆင်မပြေဘူး
3,2,hacker တွေ အန္တရာယ်ရှိတယ်
4,0,အရာအားလုံး အကောင်းဘက်ကို ဦးတည်နေတယ်
...,...,...
2606,1,သူက အဝတ်လျှော်နေတာ။
2607,1,ဒီနေ့ ကားသွားမယ်။
2608,1,သူက လမ်းဆုံမှာရပ်နေတယ်။
2609,1,မနက်ဖြန်မှာ ခရီးသွားမယ်။


In [ ]:
# Dataset to train
dataset = Dataset.from_pandas(df)

In [ ]:
# Load xlm roberta Tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
# Tokenized the text

def tokenize(batch):
  return tokenizer(
      batch['text_cleaned'],
      truncation=True,
      padding="max_length",
      max_length=128,

  )

#tokenized dataset
tokenized_ds = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2611 [00:00<?, ? examples/s]

In [ ]:
# Train/Validation split

tokenized_ds = tokenized_ds.train_test_split(test_size=0.2, seed=42)
train_ds = tokenized_ds["train"]
val_ds = tokenized_ds["test"]
train_ds

Dataset({
    features: ['label', 'text_cleaned', 'input_ids', 'attention_mask'],
    num_rows: 2088
})

In [ ]:
# Load the model
num_label= len(set(df["label"]))
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels= num_label)
model.to(device)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [ ]:
# Metrics Computation

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  acc = accuracy_score(labels, preds)
  precision, recall , f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
  return {
      'accuracy':acc,
      'precision':precision,
      'recall':recall,
      'f1':f1
  }

In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir = "./xlm_roberta_finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size = 8, # to reduce gpu usage
    per_device_eval_batch_size = 8,
    num_train_epochs = 3,
    weight_decay=0.01,
    load_best_model_at_end= True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=True if device=="cuda" else False
)

In [ ]:
# Trainer
trainer= Trainer(
    model = model,
    args = training_args,
    train_dataset= train_ds,
    eval_dataset= val_ds,
    tokenizer= tokenizer,
    compute_metrics = compute_metrics
)

/tmp/ipython-input-3008823286.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer= Trainer(


In [ ]:
# Train the model
trainer.train()

# save to drive
from google.colab import drive

drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/burmese_sentiment_xlm_roberta"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved to {save_path}")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yoonthiriaung04 (yoonthiriaung04-ucsy) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.617053,0.785851,0.799356,0.785851,0.787536
2,0.840900,0.426776,0.847036,0.846538,0.847036,0.846422
3,0.840900,0.411491,0.845124,0.844704,0.845124,0.844784


Mounted at /content/drive
Model saved to /content/drive/MyDrive/burmese_sentiment_xlm_roberta


In [3]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch

# Load model from your Google Drive path
model_path = "emilyyy04/burmese-sentiment-xlm-roberta"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Map index back to labels
id2label = {0: "positive", 1: "neutral", 2: "negative"}

# predict
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        pred_id = torch.argmax(probs, dim=1).item()
    return id2label[pred_id], probs.tolist()[0]

# Example test
text = "ကျက်စာမပေးသေးဘူး အခုထိ"
label, probabilities = predict_sentiment(text)
print(f"Predicted label: {label}")
print(f"Probabilities: {probabilities}")


Predicted label: neutral
Probabilities: [0.3042115271091461, 0.654137134552002, 0.04165137931704521]


In [5]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load model
tokenizer = AutoTokenizer.from_pretrained("emilyyy04/burmese-sentiment-xlm-roberta")
model = AutoModelForSequenceClassification.from_pretrained("emilyyy04/burmese-sentiment-xlm-roberta")

id2label = {0: "positive", 1: "neutral", 2: "negative"}

def predict(text):
    if not text.strip():
        return "Please enter text.", {}

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1).tolist()[0]
        pred_id = torch.argmax(torch.tensor(probs)).item()

    prediction = id2label[pred_id]

    # Return label and dictionary for Gradio's label component
    scores = {
        "positive": probs[0],
        "neutral": probs[1],
        "negative": probs[2],
    }

    return prediction, scores

# Create Gradio Interface
demo = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(label="Enter Burmese text", placeholder="Type here..."),
    outputs=[
        gr.Label(label="Predicted Sentiment"),
        gr.Label(label="Sentiment Scores"),
    ],
    title="Burmese Sentiment Analysis",
    description="Classify Burmese text as Positive, Neutral, or Negative using a fine-tuned XLM-RoBERTa model.",
    examples=[
        ["သူ့အပြုအမူက အရမ်းယဥ်ကျေးတယ်။"],
        ["စိတ်ပျက်တယ်"],
        ["ဒီနေ့ရာသီဥတုအေးတယ်"],
    ]
)

if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5b09b6564f4a242c0f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
